Dieses Skript erlaubt eine einfache Übersicht über Daten in AP2 und außerdem die Eingabe neuer Daten über Dialog-Felder in der Konsole.

**Schritt 1: Notebook mit eigenem Google Drive verbinden.**

In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

**Schritt 2: Benötigte Python Packages installieren.**

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install xlsxwriter
!pip install gwpy --quiet

**Schritt 3: Benötigte Python Packages in Skript importieren.**

In [ ]:
## import relevant packages

import xlsxwriter
import pandas as pd
import matplotlib
import os
import difflib

**Schritt 4: Daten laden und benötigte Spalten für die Excel-Tabelle festlegen.** Daten aus einzelnen Tabellenblättern werden zusammengeführt.

In [ ]:
from sys import breakpointhook
file=directory+"INPUT AP2/Probedaten-AP-2-DigiKAR_ModellierungMB.xlsx"

rights_list=["Amtszugehörigkeit", "Grundherrschaft", "Hochgericht", "Erbgericht", "Kirche", "Ortsliste", "Sässigkeit", "Abgabenempfänger"]
places_sheet=pd.read_excel(file, sheet_name="Ortsliste", index_col=None, dtype=str)
places_list=places_sheet["Ortsname"].values.tolist()
df_list=[]

df_Amt = pd.read_excel(file, sheet_name="Amtszugehörigkeit", dtype=str)
df_Grund = pd.read_excel(file, sheet_name="Grundherrschaft", dtype=str)
df_Hoch = pd.read_excel(file, sheet_name="Hochgericht", dtype=str)
df_Erb = pd.read_excel(file, sheet_name="Erbgericht", dtype=str)
df_Kirche = pd.read_excel(file, sheet_name="Kirche", dtype=str)
df_Saessigkeit = pd.read_excel(file, sheet_name="Sässigkeit", dtype=str)
df_Abgabenempfaenger = pd.read_excel(file, sheet_name="Abgabenempfänger", dtype=str)

df_all = pd.concat([df_Amt, df_Grund, df_Hoch, df_Erb, df_Kirche, df_Saessigkeit, df_Abgabenempfaenger], ignore_index=True, sort=False)
df_all.fillna("n/a")

rightholders_mult=[]
rightholders_combined = df_all["Rechteinhaber"].values.tolist()
for r in rightholders_combined:
  r_new_list=str(r).split(", ")
  for r_new in r_new_list:
    rightholders_mult.append(r_new)
rightholders=set(rightholders_mult)
#print(rightholders)

display(df_all)

AP2_columns=["Ortsname", "Ortslevel", "Rechtstyp", "Rechteinhaber", "Rechtsstatus", "Ausübungsort", "Abweichung", "Jahr", "nach-Jahr", "vor-Jahr", "Quelle", "Anmerkung"]

**Schritt 5: Nutzerdialog starten und Eingaben verarbeiten.**

In [ ]:
# Nutzereingabe und Verarbeitung

# Funktion zur Datenansicht
def view_data(param1):

    print("Suche Recht, Rechteinhaber oder Ort:")
    user_input=input()

    if user_input in rights_list:
      data_AP2 = pd.read_excel(file, sheet_name=user_input, index_col=None, dtype=str)
      display(data_AP2)
      new_selection(1)
    elif user_input in places_list:
      data_AP2 = df_all.loc[df_all["Ortsname"] == user_input]
      display(data_AP2)
      new_selection(1)
    elif user_input in rightholders:
      try:
          df_all['Rechteinhaber_str'] = df_all['Rechteinhaber'].map(str) 
          data_AP2 = df_all.loc[df_all["Rechteinhaber"].str.contains(user_input, na=False)]
          display(data_AP2)
          new_selection(1)
      except:
        print("Gültiger Rechteinhaber, aber kein Treffer in den Daten!")
        view_data(1)
    else:
      print("Kein passender Wert gefunden!")
      view_data(1)

# Funktion zur Dateneingabe
def input_data(param2):
    
    global new_df

    print("Neue Information")
    print("Ort:")
    user_Ort=input()
    print("Ortslevel:")
    user_Ortslevel=input()
    print("Rechtstyp:")
    user_Rechtstyp=input()
    print("Rechteinhaber:")
    user_Rechteinhaber=input()
    print("Rechtsstatus:")
    user_Rechtsstatus=input()
    print("Ausübungsort:")
    user_Ausübungsort=input()
    print("Abweichung:")
    user_Abweichung=input()
    print("Jahr:")
    user_Jahr=input()
    print("nach-Jahr:")
    user_nach_Jahr=input()
    print("vor-Jahr:")
    user_vor_Jahr=input()
    print("Quelle:")
    user_Quelle=input()
    print("Anmerkung:")
    user_Anmerkung=input()
    
    new_info_list=[user_Ort, 
                  user_Ortslevel,
                  user_Rechtstyp, 
                  user_Rechteinhaber,
                  user_Rechtsstatus, 
                  user_Ausübungsort, 
                  user_Abweichung, 
                  user_Jahr, 
                  user_nach_Jahr, 
                  user_vor_Jahr, 
                  user_Quelle, 
                  user_Anmerkung]

    new_df = pd.DataFrame(columns=AP2_columns, data=[new_info_list])
    display(new_df)
    df_list.append(new_df)
    new_selection(1)

# Funktion zum Beenden des Skripts
def end_script(param4):
  print("Skript beendet. Starten Sie das Skript für weitere Datenbearbeitung erneut!")
  
# Funktion zur Abfrage weiterer Optionen
def new_selection(param3):   

    try_again = input("0 = Schließen, 1 = neue Auswahl, 2 = Daten hinzufügen oder ändern. \n Das Schließen speichert Ihre Eingaben in einer neuen Datei.")
    try:
        try_again = int(try_again)  # non-numeric input from user could otherwise crash at this point
        if try_again == 0:
            end_script(1)
        if try_again == 1:
            view_data(1)
        if try_again == 2:
            input_data(1)
    except Exception as e:
        print(e)

# Hauptfunktion
def main():
    view_data(1)
    if len(df_list)>=1:
        # Neue Informationen in neue Excel-Datei schreiben
        fName = input("Wählen Sie einen Namen für Ihre Datei: ")
        workbook=directory+'OUTPUT AP2/'+fName+'.xlsx'
        #old_df=pd.read_excel(file, index_col=None, dtype=str)
        for df in df_list:
            df_to_write = pd.concat(df_list, ignore_index=True, sort=False)
            writer = pd.ExcelWriter(workbook, engine='xlsxwriter')
            with writer as writer:
                df_to_write.to_excel(writer, sheet_name='AP2_new')
        print("Eingabe gespeichert.")
    else:
        print("Bis zum nächsten Mal!")

if __name__ == "__main__":
    main()

Skript für das <a href="https://digikar.eu/">DigiKAR</a> Projekt im Februar 2023.

Erstellt von Monika Barget, IEG Mainz / Universität Maastricht.